In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
df = pd.DataFrame(pd.read_excel('./datasci/cleanedup/whl_2025_base.xlsx'))
df.head()

,game_id,record_id,home_team,away_team,went_ot,home_off_line,home_def_pairing,away_off_line,away_def_pairing,home_goalie,...,home_goals,away_assists,away_shots,away_xg,away_max_xg,away_goals,home_penalties_committed,home_penalty_minutes,away_penalties_committed,away_penalty_minutes
0,game_0001,record_1,thailand,pakistan,0,PP_kill_dwn,PP_kill_dwn,PP_up,PP_up,player_id_142,...,0,2,9,1.4645,0.2166,1,7,14,1,2
1,game_0001,record_2,thailand,pakistan,0,second_off,second_def,second_off,second_def,player_id_142,...,0,2,1,0.0928,0.0928,1,0,0,0,0
2,game_0001,record_19,thailand,pakistan,0,second_off,first_def,first_off,second_def,player_id_142,...,0,0,0,0.0000,0.0000,0,0,0,0,0
3,game_0001,record_11,thailand,pakistan,0,second_off,first_def,second_off,second_def,player_id_142,...,0,0,1,0.0763,0.0763,0,0,0,0,0
4,game_0001,record_10,thailand,pakistan,0,second_off,second_def,first_off,first_def,player_id_142,...,0,0,0,0.0000,0.0000,0,0,0,0,0


In [87]:
'''
IDENTIFIERS:
- game_id
- record_id

ENTITIES:
- home_team
- away_team
- home_goalie
- away_goalie

CONTEXT:
- home_off_line
- away_off_line
- home_def_pairing
- away_def_pairing
- went_ot

OUTCOMES:
- home_goals
- away_goals
- home_shots
- away_shots
- home_penalties_committed
- away_penalties_committed

DERIVED METRICS:
- home_xg
- away_xg
- home_max_xg
- away_max_xg
'''

'\nIDENTIFIERS:\n- game_id\n- record_id\n\nENTITIES:\n- home_team\n- away_team\n- home_goalie\n- away_goalie\n\nCONTEXT:\n- home_off_line\n- away_off_line\n- home_def_pairing\n- away_def_pairing\n- went_ot\n\nOUTCOMES:\n- home_goals\n- away_goals\n- home_shots\n- away_shots\n- home_penalties_committed\n- away_penalties_committed\n\nDERIVED METRICS:\n- home_xg\n- away_xg\n- home_max_xg\n- away_max_xg\n'

In [88]:
sum_cols = [
    "home_goals", "away_goals",
    "home_shots", "away_shots",
    "home_xg", "away_xg",
    "home_assists", "away_assists",
    "home_penalties_committed", "away_penalties_committed",
    "home_penalty_minutes", "away_penalty_minutes"
]
first_cols = [
    "home_team",
    "away_team",
    "went_ot"
]
agg_dict = {}

for col in sum_cols:
    agg_dict[col] = "sum"

for col in first_cols:
    agg_dict[col] = "first"
games = (
    df
    .groupby("game_id", as_index=False)
    .agg(agg_dict)
)
games.shape

(1312, 16)

In [89]:
games["home_score"] = games["home_goals"]
games["away_score"] = games["away_goals"]

games["goal_diff"] = games["home_score"] - games["away_score"]
games["total_goals"] = games["home_score"] + games["away_score"]

games["shot_diff"] = games["home_shots"] - games["away_shots"]
games["total_shots"] = games["home_shots"] + games["away_shots"]

games


,game_id,home_goals,away_goals,home_shots,away_shots,home_xg,away_xg,home_assists,away_assists,home_penalties_committed,...,away_penalty_minutes,home_team,away_team,went_ot,home_score,away_score,goal_diff,total_goals,shot_diff,total_shots
0,game_0001,1,3,21,24,2.8231,2.7516,2,6,8,...,12,thailand,pakistan,0,1,3,-2,4,-3,45
1,game_0002,3,2,31,27,3.4075,3.0509,4,4,9,...,12,france,morocco,1,3,2,1,5,4,58
2,game_0003,4,3,42,26,4.8049,2.3901,6,6,6,...,24,peru,iceland,1,4,3,1,7,16,68
3,game_0004,0,1,19,16,1.6322,1.6166,0,2,6,...,10,mongolia,saudi_arabia,0,0,1,-1,1,3,35
4,game_0005,4,1,32,42,3.1535,4.7456,7,1,9,...,12,usa,guatemala,0,4,1,3,5,-10,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,game_1308,2,0,13,24,1.2708,2.4745,3,0,7,...,10,switzerland,france,0,2,0,2,2,-11,37
1308,game_1309,5,4,29,29,4.2460,2.3279,9,5,2,...,24,netherlands,south_korea,0,5,4,1,9,0,58
1309,game_1310,1,3,21,19,3.0211,2.1164,1,5,6,...,8,uk,uae,0,1,3,-2,4,2,40
1310,game_1311,3,0,18,24,2.4804,1.9921,5,0,6,...,13,brazil,rwanda,0,3,0,3,3,-6,42


In [90]:
home_games = games.copy()

home_games["team"] = home_games["home_team"]
home_games["opponent"] = home_games["away_team"]

home_games["goals_for"] = home_games["home_score"]
home_games["goals_against"] = home_games["away_score"]

home_games["shots_for"] = home_games["home_shots"]
home_games["shots_against"] = home_games["away_shots"]

home_games["xg_for"] = home_games["home_xg"]
home_games["xg_against"] = home_games["away_xg"]

home_games["is_home"] = 1

In [91]:
away_games = games.copy()

away_games["team"] = away_games["away_team"]
away_games["opponent"] = away_games["home_team"]

away_games["goals_for"] = away_games["away_score"]
away_games["goals_against"] = away_games["home_score"]

away_games["shots_for"] = away_games["away_shots"]
away_games["shots_against"] = away_games["home_shots"]

away_games["xg_for"] = away_games["away_xg"]
away_games["xg_against"] = away_games["home_xg"]

away_games["is_home"] = 0

In [92]:
'''team_games = pd.concat([home_games, away_games], ignore_index=True)
team_games["goal_diff"] = team_games["goals_for"] - team_games["goals_against"]

team_games["win"] = (team_games["goal_diff"] > 0).astype(int)
team_games["loss"] = (team_games["goal_diff"] < 0).astype(int)
team_season = (
    team_games
    .groupby("team", as_index=False)
    .agg(
        games_played = ("team", "count"),
        wins = ("win", "sum"),
        losses = ("loss", "sum"),
        goals_for = ("goals_for", "sum"),
        goals_against = ("goals_against", "sum"),
        shots_for = ("shots_for", "sum"),
        shots_against = ("shots_against", "sum"),
        xg_for = ("xg_for", "sum"),
        xg_against = ("xg_against", "sum"),
        avg_goal_diff = ("goal_diff", "mean"),
        home_games = ("is_home", "sum")
    )
)
team_season["goals_per_game"] = team_season["goals_for"] / team_season["games_played"]
team_season["goals_against_per_game"] = team_season["goals_against"] / team_season["games_played"]

team_season["shot_diff"] = team_season["shots_for"] - team_season["shots_against"]
team_season["xg_diff"] = team_season["xg_for"] - team_season["xg_against"]
team_season["win_pct"] = team_season["wins"] / team_season["games_played"]
team_season["loss_pct"] = team_season["losses"] / team_season["games_played"]

team_season
team_season.sort_values("loss_pct", ascending=False).head()


,team,games_played,wins,losses,goals_for,goals_against,shots_for,shots_against,xg_for,xg_against,avg_goal_diff,home_games,goals_per_game,goals_against_per_game,shot_diff,xg_diff,win_pct,loss_pct
12,mongolia,82,27,55,179,244,2018,2314,187.5990,268.4970,-0.792683,41,2.182927,2.975610,-296,-80.8980,0.329268,0.670732
10,kazakhstan,82,30,52,193,248,1986,1955,209.8053,231.4215,-0.670732,41,2.353659,3.024390,31,-21.6162,0.365854,0.634146
21,rwanda,82,30,52,202,265,2126,2150,223.8094,256.1218,-0.768293,41,2.463415,3.231707,-24,-32.3124,0.365854,0.634146
16,oman,82,35,47,241,283,2176,2287,244.9371,279.4926,-0.512195,41,2.939024,3.451220,-111,-34.5555,0.426829,0.573171
30,usa,82,35,47,240,287,2171,2400,247.3928,281.6973,-0.573171,41,2.926829,3.500000,-229,-34.3045,0.426829,0.573171


In [93]:

team_games["shot_diff"] = team_games["shots_for"] - team_games["shots_against"]
team_games["xg_diff"] = team_games["xg_for"] - team_games["xg_against"]
team_games["goal_diff"] = team_games["goals_for"] - team_games["goals_against"]
team_games["xg_overperf"] = team_games["goals_for"] - team_games["xg_for"]

team_games["power_score"] = (
    0.40 * team_games["xg_diff"] +
    0.30 * team_games["shot_diff"] +
    0.20 * team_games["goal_diff"] +
    0.10 * team_games["xg_overperf"]

)

HOME_ADVANTAGE = 0.25
team_games["adj_power_score"] = (
    team_games["power_score"] - HOME_ADVANTAGE * team_games["is_home"]
)

# 4. Average by team
power_rankings = (
    team_games
    .groupby("team", as_index=False)
    .agg(
        games_played=("adj_power_score", "size"),
        avg_power=("adj_power_score", "mean"),
        median_power=("adj_power_score", "median"),
        shot_diff_pg=("shot_diff", "mean"),
        xg_diff_pg=("xg_diff", "mean"),
        goal_diff_pg=("goal_diff", "mean"),
    )
)

power_rankings = (
    power_rankings
    .sort_values("avg_power", ascending=False)
    .reset_index(drop=True)
)

power_rankings["power_rank"] = power_rankings.index + 1

power_rankings


,team,games_played,avg_power,median_power,shot_diff_pg,xg_diff_pg,goal_diff_pg,power_rank
0,pakistan,82,1.938734,1.558995,5.719512,0.624895,0.621951,1
1,peru,82,1.357824,1.984415,3.780488,0.361174,0.951220,2
2,thailand,82,1.143980,0.420225,2.670732,0.891161,0.560976,3
3,south_korea,82,1.121057,1.074210,4.012195,0.105560,-0.170732,4
4,mexico,82,0.849163,0.639005,2.829268,0.502351,-0.207317,5
5,france,82,0.708414,0.564630,2.548780,0.396499,-0.292683,6
6,uk,82,0.696868,0.667860,2.012195,0.442818,0.402439,7
7,netherlands,82,0.630223,0.770180,1.292683,0.496285,0.841463,8
8,new_zealand,82,0.606059,0.692690,2.536585,0.035772,-0.085366,9
9,india,82,0.531437,0.641865,2.012195,0.017117,0.341463,10
